In [11]:
import warnings
import sys
warnings.simplefilter('ignore')
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
from tqdm import tqdm 

In [2]:
import pandas as pd
bad = pd.read_csv('../data/clean/cleaned_bad.csv')
normal=pd.read_csv('../data/clean/cleaned_normal.csv')
# hiniku_=pd.read_csv('../data/hiniku/hiniku_with_word.csv')
hiniku = pd.read_csv("../data/clean/cleaned_hiniku.csv")
df = pd.read_csv('../data/clean/mount_all_data.csv')
# df.drop('comment_id', axis=1, inplace=True)

In [3]:
badwordlist = pd.read_csv('../data/clean/mount_bad_list.csv')
from janome.tokenizer import Tokenizer

tokenizer=Tokenizer()
badwordlist_reading = []
for word  in badwordlist.bad.values:
    text=""
    for t in tokenizer.tokenize(word):
        if t.reading=="*":
            pass
        else:
            text+=t.reading
    if len(text)>=1:
        badwordlist_reading.append(text)
# pd.DataFrame(badwordlist_reading).to_csv('../data/clean/badwordlist_reading.csv', index=False)

In [4]:
# badwordlist

In [333]:
from sklearn.svm import SVR
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


def train_and_pred(df, threshold=0.5):
    df = df.dropna()
    # testdf.target = testdf.target.apply(lambda x: 0 if x==0 else 1)#x==0(ニュートラル)の時0, その他は1
    if len(df.target.unique())>2:
        df.target = df.target.apply(lambda x: 1 if x==1 else 0)#x==1(悪口)の時1, その他は0 [0:悪口, 1:その他]
    # testdf.target = testdf.target.apply(lambda x: 0 if x==2 else 1)#x==2(皮肉)の時0, その他は1
    X = df.drop(["target"],axis=1)#.drop('content', axis=1)
    y=df["target"]
    x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)    

    svm = SVR()
    svm.fit(x_train.drop(['content'], axis=1), y_train)
    pred_train_unadjust = svm.predict(x_train.drop(['content'], axis=1))
    pred_test_unadjust = svm.predict(x_test.drop(['content'], axis=1))

    print("train")

    pred_train = [1 if x>threshold else 0 for x in pred_train_unadjust]
    pred_test = [1 if x>threshold else 0 for x in pred_test_unadjust]
    print("f1: " ,f1_score(y_train, pred_train))
    display(create_2x2_matrix(y_train, pred_train))
    # print_2x2_miss_pred(y_train, pred_train,x_train)
    print('---------------------------------')
    print('test')

    print("f1: ",f1_score(y_test, pred_test))
    display(create_2x2_matrix(y_test, pred_test))
    train_miss = print_2x2_miss_pred(true_y=y_train,pred_y=pred_train,df=x_train)
    test_miss =  print_2x2_miss_pred(true_y=y_test, pred_y=pred_test,df=x_test)
    return {
        "miss_train_df":train_miss, 
        "miss_test_df":test_miss,
        "x_train":x_train,
        "y_train":y_train,
        "x_test":x_test,
        "y_test":y_test,
        "pred_train":pred_train,
        "pred_test":pred_test,
        "model":svm,
        "pred_train_un":pred_train_unadjust,
        "pred_test_un":pred_test_unadjust,
    }

In [6]:
def create_2x2_matrix(true_y, pred_y):
    matrix = np.zeros(4)

    for t,p in zip(true_y, pred_y):
        if t==0:
            if p==0:matrix[0]+=1
            if p==1:matrix[1]+=1
        elif t==1:
            if p==0:matrix[2]+=1
            if p==1:matrix[3]+=1
    return matrix.reshape(2,2)


In [7]:
def print_2x2_miss_pred(true_y, pred_y,df, test=True):
    def print_str_label(num):
        if num==1:return "悪口"
        if num==0:return "皮肉or その他"
        return np.nan

    idx_arr = []
    pred_judge=[]
    true_judge=[]
    col=["content", "pred_idx", "pred_label","true_idx", "true_label"]
    returndf = pd.DataFrame(columns=col)
    for i in range(len(df)):
        if pred_y[i]!=true_y.values[i]:
            tmpdf = pd.DataFrame(np.array([df.iloc[i].content, pred_y[i], print_str_label(pred_y[i]),\
                                            true_y.values[i], print_str_label(true_y.values[i])]).reshape(1,-1), \
                                              columns=col)
            returndf = pd.concat([returndf, tmpdf])
    return returndf


def print_2x2_correct_pred(true_y, pred_y,df, test=True):
    def print_str_label(num):
        if num==1:return "悪口"
        if num==0:return "皮肉or その他"
        return np.nan

    idx_arr = []
    pred_judge=[]
    true_judge=[]
    col=["content", "pred_idx", "pred_label","true_idx", "true_label"]
    returndf = pd.DataFrame(columns=col)
    for i in range(len(df)):
        if pred_y[i]==true_y.values[i]:
            tmpdf = pd.DataFrame(np.array([df.iloc[i].content, pred_y[i], print_str_label(pred_y[i]),\
                                            true_y.values[i], print_str_label(true_y.values[i])]).reshape(1,-1), \
                                              columns=col)
            returndf = pd.concat([returndf, tmpdf])
    return returndf

In [8]:
def create_wakati_content(text):
    wakati = []
    for t in tokenizer.tokenize(text):
        wakati.append(t.surface)
    return wakati

In [13]:
from gensim.models import Word2Vec
from tqdm import tqdm
# # data = pd.read_csv('../data/clean/mount_bad_list.csv')
# data = pd.read_csv("../data/clean/mount_all_data.csv")
# data = data[data.type==1]
wakati_arr = []
# for r in tqdm(data.iterrows()):
#     wakati = create_wakati_content(r[1].content)
#     wakati_arr.append(wakati)
# model = Word2Vec(wakati_arr, sg=1, size=100, window=5, min_count=1)


In [15]:
# data = pd.read_csv('../data/some_comment_data.csv')

# for r in tqdm(data.iterrows()):
#     wakati = create_wakati_content(r[1].content)
#     wakati_arr.append(wakati)
# model = Word2Vec(wakati_arr, sg=1, size=100, window=5, min_count=1)
# model.save("./word2vecModel/200000.model")


In [16]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load("./word2vecModel/100000.model")

In [206]:
'''
悪口ワードの含有
皮肉ワードの含有
ポジティブワードの含有

[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [形態素ごとに現れる悪口数]
[皮肉単語が含まれているかのフラグ],[形態素ごとに現れる皮肉数]
[ポジティブな単語が含まれているかのフラグ], [形態素ごとに現れるポジティブ単語数]

'''
import pandas as pd
# df = bad.copy()
# df = pd.concat([hiniku_, normal])
# df = pd.concat([bad, df])
import sys
import numpy as np
from janome.tokenizer import Tokenizer
from tqdm import tqdm
import gensim

df = pd.read_csv('../data/clean/mount_all_data.csv')
df.drop('comment_id', axis=1, inplace=True)
tokenizer = Tokenizer()
#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_word_count(text, wordlist):
    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in wordlist:
            count+=1
    for w in wordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


def get_order_label(text):
    for t in tokenizer.tokenize(text):
        if t.infl_form[:2]=="命令":
            return 1
    return 0



def cos_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


'''
###ここを実行することでword2vecのモデル作成
from gensim.models import Word2Vec
data = pd.read_csv('../data/some_comment_data.csv')
newdata=pd.concat([pd.DataFrame(data.content), pd.DataFrame(df.content)])
wakati_arr = []
for r in tqdm(newdata.iterrows()):
    wakati = create_wakati_content(r[1].content)
    wakati_arr.append(wakadti)
model = Word2Vec(wakati_arr, sg=1, size=100, window=5, min_count=1)
'''

def calc_cosine_similary(text,model,debug=False):
    text_list = []
    scores = {}

    for t in tokenizer.tokenize(text):
        text_list.append(t.surface)
    for word in text_list:
        try:
            vector1 = model.wv[word]
            scores[word] = []
            for bad in bad_list:
                try:
                    vector2 = model.wv[bad]
                    score = cos_similarity(vector1,vector2)
                    scores[word].append(score)
                except :
                    pass
        except:
            pass
    all_scores = []
    for scores in scores.values():
        all_scores +=scores
    all_scores= np.array(all_scores)
    typicalV = np.median(all_scores, axis=0)

    if debug:
        return scores
    return typicalV


def mophologicize(text):
    words = []
    for t in tokenizer.tokenize(text):
        words.append(t.surface)    
    return words


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_features(df, badwordlist, positivelist=np.nan, hinikulist=np.nan):
#     col=["content","mophologics_num", "bad_per_mophologic", "simple_bad_flg","target", "hiniku_per_mophologic",\
#          "simple_hiniku_flg", "positive_per_mophologic", "simple_positive_flg"]
#     import gensim
#     PATH="./gensimmodel/word2vec.gensim.model"
#     model = gensim.models.Word2Vec.load(PATH)
    col=["content","mophologics_num", "bad_count_in_mophologic", "simple_bad_flg","target","order","mophologic_content"]

    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in tqdm(df.iterrows()):
        text = str(r[1].content)
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        badcount,badflg = get_word_count(text, badwordlist)
        orderlabel = get_order_label(text)
        mophologic = mophologicize(text)
#         hinikucount, hinikuflg = get_word_count(text, hinikulist)
#         positivecount, positiveflg = get_word_count(text, positivelist)
#         newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target,\
#                                        hinikucount, hinikuflg, positivecount,\
#                                        positiveflg]).reshape(1,-1),columns=col)
        newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target, orderlabel,mophologic]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
#     basedf["comment_id"] = basedf.index
    basedf["bad_count_in_mophologic"] = basedf.apply(lambda x: float(x["bad_count_in_mophologic"]), axis=1)
    basedf["mophologics_num"] = basedf.apply(lambda x: float(x["mophologics_num"]), axis=1)
    basedf["order"] = basedf.order.apply(lambda x: int(x))
#     basedf["cosine_similar"] = basedf.cosine_similar.apply(lambda x: float(x))
    
    return basedf
df =make_features(df, list(badwordlist.bad.values))
df = df.fillna(0.0)
df = df.reset_index().drop('index', axis=1)
df["id"] = df.index
display(df.head(2))
print(df.shape)


1929it [00:15, 121.15it/s]


,content,mophologics_num,bad_count_in_mophologic,simple_bad_flg,target,order,mophologic_content,id
0,なにこの糞ゲー,4.0,1.0,1,1,0,"[なに, この, 糞, ゲー]",0
1,ちょっと蛇足気味だな・・・,8.0,1.0,1,1,0,"[ちょっと, 蛇足, 気味, だ, な, ・, ・, ・]",1


(1929, 8)


In [18]:
def create_bad_word_list():
    b_l=[]
    #wor2vecの辞書にない単語の除外
    for word in badwordlist.bad.values:
        try:
            a = model.wv[word]
            b_l.append(word)
        except:
            pass
    return b_l

In [341]:

col=[]
for i in range(100):
    col.append("f_"+str(i))
def create_mean_vector_data(df):

    sandbox=pd.DataFrame(columns=col)
    for r in tqdm(df.iterrows()):
        text = r[1].mophologic_content
        vec_list=[]
        for t in text:
                try:
                    vec = model.wv[t]
                    vec_list.append(vec)
                except:
                    vec_list.append(np.zeros(100))
        vectors = pd.DataFrame(np.array(vec_list), columns=col)
        tmp = pd.DataFrame(pd.DataFrame(vectors, columns=col).mean(axis=0)).T
        sandbox = pd.concat([sandbox, tmp])
        

    sandbox.insert(0, "target",df.target.values)
    sandbox.insert(1,"content",df.content.values)
    return sandbox

In [313]:
res1=train_and_pred(sandbox,threshold=0.23)sandbox

train


In [344]:
# create_mean_vector_data(df)

,target,content,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,...,f_90,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99
0,1,なにこの糞ゲー,0.171568,0.094425,-0.243519,-0.052515,-0.070759,0.411506,-0.151271,0.049143,...,-0.192252,0.003019,0.412266,-0.066477,0.105831,-0.042882,0.257228,0.351142,-0.161795,-0.213026
0,1,ちょっと蛇足気味だな・・・,-0.043816,0.080323,-0.144619,-0.005080,0.173006,0.395386,-0.056568,0.039275,...,-0.263620,-0.129886,0.359296,-0.119541,0.160785,-0.061619,0.276749,0.451937,-0.257912,-0.222234
0,1,蛇足は同意,0.208156,0.006126,-0.132161,-0.044880,0.013240,0.201669,-0.085165,0.097970,...,0.037020,-0.047831,0.301364,0.015739,0.187576,0.087266,0.031497,0.311311,-0.030625,-0.046450
0,1,蛇足っていってるヤツはJazzのライブを聞いたことがない奴,0.192251,0.026909,-0.158000,-0.141699,0.061730,0.190829,-0.010910,0.197090,...,-0.072041,-0.172346,0.355718,0.091098,0.022344,-0.078946,0.181374,0.315143,-0.089571,0.001473
0,1,蛇足っつう～かﾗｲﾌﾞｱﾚﾝｼﾞのお約束みたいなもんだろ,0.161773,-0.037911,-0.236227,-0.164525,-0.029755,0.214892,-0.170693,-0.051191,...,-0.127900,-0.149476,0.190359,0.028135,0.175896,-0.048945,0.095601,0.192192,-0.305131,-0.225195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,動画もいいな,0.260085,-0.001982,-0.463291,-0.092470,-0.009754,0.158714,-0.216898,0.039007,...,-0.214941,-0.445160,0.296062,-0.037287,0.100892,-0.101708,0.252462,0.409400,-0.169833,-0.137619
0,0,3桁記念かきこ,0.196400,0.001033,-0.191610,-0.135905,0.071396,0.116956,-0.148527,0.038829,...,-0.106778,-0.094532,0.104275,0.090057,0.201584,-0.028431,-0.109992,0.199174,-0.179325,-0.132668
0,0,鳥肌やばい,0.277268,0.229609,-0.306203,-0.279015,0.150056,0.260009,-0.122000,0.095843,...,-0.360716,-0.207681,0.392508,-0.087436,-0.004210,-0.171299,0.110205,0.402899,-0.220459,-0.147590
0,0,兄弟愛に泣いた,0.189268,0.067153,-0.251199,-0.267033,0.084082,0.251470,-0.121412,0.144522,...,-0.222742,-0.417797,0.305011,0.048205,-0.001763,-0.026853,0.152260,0.296517,-0.269014,-0.083580


In [352]:
def create_mean_cosine_sim_data(df):
    b_l=create_bad_word_list()
    base = pd.DataFrame(columns=np.array(b_l))
    
    for r in tqdm(df.iterrows()):
        text = r[1].mophologic_content
        ave_list=[]

        for word in b_l:
            try:
                v1 = model.wv[word]
                cos_list = []
                for t in text:
                        v2=model.wv[t]
                        cos_list.append(cos_similarity(v1,v2))
                        if word not in b_l:
                            b_l.append(word)
            except:
                pass
            ave_list.append(np.array(cos_list).mean())
        tmp=pd.DataFrame(np.array(ave_list).reshape(1,-1), columns=b_l)
        base=pd.concat([base, tmp])
        
    base["target"] =df.target.values
    base["content"] = df.content.values
    base = base.loc[:,base.columns[::-1]]

    return base

In [182]:
mean_sim_df = create_mean_cosine_sim_data(df)
mean_sim_df.head(3)

1929it [00:56, 34.06it/s]


,content,target,はげ,キモオタ,バカー,きしょい,クソゲー,クソムシ,きらい,gm,...,シカト,ゆとり,ダサい,死ね,KY,イタイ,ばっか,ヘタ,蛇足,糞
0,なにこの糞ゲー,1,0.737712,0.585149,0.553066,0.503287,0.660441,0.685455,0.790680,0.616264,...,0.596774,0.808434,0.770303,0.681981,0.660041,0.673967,0.833161,0.694381,0.698025,0.844980
0,ちょっと蛇足気味だな・・・,1,0.670004,0.588808,0.553636,0.499966,0.630143,0.643968,0.664984,0.602853,...,0.588668,0.682783,0.706425,0.545869,0.592983,0.637162,0.679401,0.664682,0.654728,0.703167
0,蛇足は同意,1,0.833095,0.794032,0.705397,0.689864,0.834422,0.844973,0.823597,0.822670,...,0.774778,0.816561,0.865877,0.677959,0.799566,0.823735,0.757569,0.806733,0.857879,0.779104


In [183]:
res2 = train_and_pred(mean_sim_df,threshold=0.2)

train
f1:  0.7031914893617021


array([[273., 481.],
       [ 77., 661.]])

---------------------------------
test
f1:  0.6967213114754097


array([[ 56., 124.],
       [ 24., 170.]])

In [354]:
def create_the_highest_cosine_sim_data(df):
    b_l = create_bad_word_list()
    base = pd.DataFrame(columns=np.array(b_l))
    
    for r in tqdm(df.iterrows()):
        text = r[1].mophologic_content
        highest_list=[]

        for word in b_l:
            try:
                v1 = model.wv[word]
                cos_list = []
                for t in text:
                        v2=model.wv[t]
                        cos_list.append(cos_similarity(v1,v2))
                        if word not in b_l:
                            b_l.append(word)
            except:
                pass
            try:
                highest_list.append(np.array(cos_list).max())
            except:
                highest_list.append(np.nan)
        tmp=pd.DataFrame(np.array(highest_list).reshape(1,-1), columns=b_l)
        base=pd.concat([base, tmp])
        
    base["target"] =df.target.values
    base["content"] = df.content.values
    base = base.loc[:,base.columns[::-1]]
    return base

# high_sim_df = create_the_highest_cosine_sim_data(df)
# high_sim_df.head(2)

In [185]:
res3 = train_and_pred(high_sim_df,threshold=0.26)

train
f1:  0.7386489479512736


array([[353., 401.],
       [ 71., 667.]])

---------------------------------
test
f1:  0.7337526205450734


array([[ 72., 108.],
       [ 19., 175.]])

In [186]:
res1["df"].shape,res2["df"].shape,res2["df"].shape

((1929, 103), (1866, 155), (1866, 155))

In [209]:
valid_id = list(res2["df"].id.values)


In [224]:
newdf = pd.DataFrame(columns=df.columns)
for i in tqdm(range(len(df))):
    if i in valid_id:
        tmp = pd.DataFrame(df.iloc[i]).T
        newdf = pd.concat([newdf, tmp])

100%|██████████| 1929/1929 [00:07<00:00, 267.30it/s]


In [355]:
df1=create_mean_vector_data(newdf)
df2=create_the_highest_cosine_sim_data(newdf)
df3=create_mean_cosine_sim_data(newdf)

1866it [00:06, 305.73it/s]
1866it [00:42, 43.45it/s]
1866it [00:45, 40.99it/s]


In [414]:
df3

,content,target,はげ,キモオタ,バカー,きしょい,クソゲー,クソムシ,きらい,gm,...,シカト,ゆとり,ダサい,死ね,KY,イタイ,ばっか,ヘタ,蛇足,糞
0,なにこの糞ゲー,1,0.737712,0.585149,0.553066,0.503287,0.660441,0.685455,0.790680,0.616264,...,0.596774,0.808434,0.770303,0.681981,0.660041,0.673967,0.833161,0.694381,0.698025,0.844980
0,ちょっと蛇足気味だな・・・,1,0.670004,0.588808,0.553636,0.499966,0.630143,0.643968,0.664984,0.602853,...,0.588668,0.682783,0.706425,0.545869,0.592983,0.637162,0.679401,0.664682,0.654728,0.703167
0,蛇足は同意,1,0.833095,0.794032,0.705397,0.689864,0.834422,0.844973,0.823597,0.822670,...,0.774778,0.816561,0.865877,0.677959,0.799566,0.823735,0.757569,0.806733,0.857879,0.779104
0,蛇足っていってるヤツはJazzのライブを聞いたことがない奴,1,0.706022,0.607392,0.528578,0.500326,0.686353,0.686747,0.703343,0.653967,...,0.624809,0.752642,0.744694,0.605136,0.622628,0.693890,0.735393,0.672292,0.700866,0.706526
0,蛇足っつう～かﾗｲﾌﾞｱﾚﾝｼﾞのお約束みたいなもんだろ,1,0.725199,0.641709,0.597139,0.570940,0.677702,0.670543,0.694109,0.645686,...,0.637345,0.738736,0.731107,0.628153,0.641106,0.709788,0.694881,0.699089,0.697893,0.721380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,動画もいいな,0,0.626146,0.517597,0.436151,0.419381,0.591118,0.590284,0.692273,0.561418,...,0.538685,0.716280,0.691977,0.615649,0.524166,0.610222,0.740473,0.604174,0.612987,0.712143
0,3桁記念かきこ,0,0.725750,0.689852,0.627817,0.582375,0.753787,0.745490,0.731255,0.744310,...,0.687163,0.720766,0.793534,0.541588,0.676400,0.746185,0.662057,0.735293,0.770483,0.659834
0,鳥肌やばい,0,0.776060,0.602311,0.630515,0.517956,0.692731,0.695167,0.850130,0.658046,...,0.635986,0.793466,0.815024,0.598607,0.697212,0.682221,0.814320,0.762895,0.725228,0.831501
0,兄弟愛に泣いた,0,0.693602,0.580414,0.524677,0.441396,0.677969,0.655139,0.710792,0.646196,...,0.603326,0.741409,0.732368,0.559652,0.590555,0.685026,0.701540,0.681587,0.689233,0.692915


In [356]:
res1 = train_and_pred(df1)
res2 = train_and_pred(df2)
res3 = train_and_pred(df3)

train
f1:  0.7496598639455783


array([[573., 181.],
       [187., 551.]])

---------------------------------
test
f1:  0.723404255319149


array([[134.,  46.],
       [ 58., 136.]])

train
f1:  0.6666666666666666


...

array([[142.,  38.],
       [ 72., 122.]])

train
f1:  0.636429085673146


array([[545., 209.],
       [296., 442.]])

---------------------------------
test
f1:  0.6382978723404256


array([[118.,  62.],
       [ 74., 120.]])

In [417]:
def ensemble(res1, res2, res3,threshold=0.5):
    train_ensemble = pd.DataFrame(np.array([res1["pred_train_un"],res2["pred_train_un"],res2["pred_train_un"]]).sum(axis=0)/3,columns=["pred"])
    test_ensemble = pd.DataFrame(np.array([res1["pred_test_un"],res2["pred_test_un"],res2["pred_test_un"]]).sum(axis=0)/3,columns=["pred"])
    train_ensemble = train_ensemble.pred.apply(lambda x: 1 if x>threshold else 0)
    test_ensemble = test_ensemble.pred.apply(lambda x: 1 if x>threshold else 0)
    train_score = f1_score(res1['y_train'], train_ensemble)
    test_score = f1_score(res1['y_test'], test_ensemble)
    return {"train_score":train_score,
            "test_score":test_score,
            "pred_train":train_ensemble,
            "pred_test":test_ensemble
           }

In [418]:
res=ensemble(res1,res2,res3)
print("train:")
print(res["train_score"])
print('test:')
print(res["test_score"])

train:
0.7402689313517339
test:
0.7645429362880886


In [416]:
create_2x2_matrix(true_y=res1["y_train"], pred_y=tr)

TypeError: 'numpy.float64' object is not iterable

In [411]:
line_score = 0.7
for threshold in np.arange(0,1,0.01):
    train_s, test_s=ensemble(res1,res2,res3,threshold)
    if test_s>line_score:
        line_score=test_s
        print(threshold)
        print(train_s, test_s)

0.08
0.68700140778977 0.7007299270072994
0.09
0.6902738432483475 0.7047970479704797
0.1
0.6955280685061845 0.7050092764378479
0.11
0.6991869918699186 0.7063197026022305
0.12
0.7032227032227033 0.7076350093109869
0.13
0.7090820786789704 0.7102803738317756
0.16
0.7194458189015339 0.7137404580152672
0.17
0.7219463753723933 0.7153846153846154
0.2
0.7328940699442473 0.7173489278752436
0.21
0.7380830343413635 0.7215686274509804
0.22
0.739556472408458 0.7218934911242605
0.23
0.7419354838709676 0.7261904761904762
0.24
0.742289597490852 0.7334669338677355
0.25
0.7445795875198309 0.7413441955193483
0.26
0.746268656716418 0.7438016528925618
0.27
0.7508090614886731 0.7463312368972747
0.31
0.7655016910935737 0.7516198704103672
0.32
0.7666856816885339 0.7571115973741795
0.34
0.7745327102803737 0.7589285714285714
0.35000000000000003
0.7796810395747193 0.7629796839729119
0.36
0.7785714285714286 0.767123287671233
0.47000000000000003
0.7576374745417516 0.7688172043010753


In [33]:
# testdf = base.copy()
# testdf = testdf.dropna()
# # testdf.target = testdf.target.apply(lambda x: 0 if x==0 else 1)#x==0(ニュートラル)の時0, その他は1
# testdf.target = testdf.target.apply(lambda x: 0 if x==1 else 1)#x==1(悪口)の時0, その他は1 [0:悪口, 1:その他]
# # testdf.target = testdf.target.apply(lambda x: 0 if x==2 else 1)#x==2(皮肉)の時0, その他は1
# testdf.target.unique()
# testdf.target.unique()
# X = testdf.drop("target",axis=1)#.drop('content', axis=1)
# y=testdf.target


In [34]:
# '''
# 1対1で予測をする時
# '''

# from sklearn.model_selection import train_test_split

# from sklearn.metrics import f1_score
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# def vectorize(x_train,x_test, vec):
#     x_train_vec = vec.fit_transform(x_train.content)
#     x_test_vec = vec.transform(x_test.content)
    
#     x_train_vec = pd.DataFrame(x_train_vec.todense())
#     x_test_vec = pd.DataFrame(x_test_vec.todense())
#     return x_train_vec,x_test_vec


# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [35]:
from sklearn.svm import SVR

def train_and_pred(df, threshold=0.5):
    df.insert(0,"id",df.index)
    
    df = df.dropna()
    # testdf.target = testdf.target.apply(lambda x: 0 if x==0 else 1)#x==0(ニュートラル)の時0, その他は1
    df.target = df.target.apply(lambda x: 1 if x==1 else 0)#x==1(悪口)の時1, その他は0 [0:悪口, 1:その他]
    # testdf.target = testdf.target.apply(lambda x: 0 if x==2 else 1)#x==2(皮肉)の時0, その他は1
#     df.target.unique()
#     df.target.unique()
    X = df.drop("target",axis=1)#.drop('content', axis=1)
    y=df[["target", "id"]]
    
    
    svm = SVR()
    svm.fit(x_train.drop(['content',"id"], axis=1), y_train.target)
    pred_train = svm.predict(x_train.drop(['content',"id"], axis=1))
    pred_test = svm.predict(x_test.drop(['content',"id"], axis=1))

    print("train")

    pred_train = [1 if x>threshold else 0 for x in pred_train]
    pred_test = [1 if x>threshold else 0 for x in pred_test]
    print("f1: " ,f1_score(y_train, pred_train))
    display(create_2x2_matrix(y_train, pred_train))
    # print_2x2_miss_pred(y_train, pred_train,x_train)
    print('----------------------------')
    print('test')

    print("f1: ",f1_score(y_test, pred_test))
    display(create_2x2_matrix(y_test, pred_test))
    train_miss = print_2x2_miss_pred(y_train, pred_train,x_train)
    test_miss =  print_2x2_miss_pred(true_y=y_test, pred_y=pred_test,df=x_test)
    return {
        "miss_train_df":train_miss, 
        "miss_test_df":test_miss,
        "x_train":x_train,
        "y_train":y_train,
        "x_test":x_test,
        "y_test":y_test,
        "pred_train":pred_train,
        "pred_test":pred_test
    }



In [36]:
# from sklearn.svm import SVR
# svm = SVR()

# # svm.fit(x_train_vec, y_train)
# # pred_train = svm.predict(x_train_vec)
# # pred_test = svm.predict(x_test_vec)


# # svm.fit(x_train.drop(['content',"cosine_similary"], axis=1), y_train)
# # pred_train = svm.predict(x_train.drop(['content',"cosine_similary"], axis=1))
# # pred_test = svm.predict(x_test.drop(['content',"cosine_similary"], axis=1))


# svm.fit(x_train.drop(['content'], axis=1), y_train)
# pred_train = svm.predict(x_train.drop(['content'], axis=1))
# pred_test = svm.predict(x_test.drop(['content'], axis=1))

# print("train")

# pred_train = [1 if x>0.899 else 0 for x in pred_train]
# pred_test = [1 if x>0.899 else 0 for x in pred_test]
# print(f1_score(y_train, pred_train))
# display(create_2x2_matrix(y_train, pred_train))
# # print_2x2_miss_pred(y_train, pred_train,x_train)
# print('------------------------------------------------------------------')
# print('test')

# print(f1_score(y_test, pred_test))
# display(create_2x2_matrix(y_test, pred_test))
# # print_2x2_miss_pred(true_y=y_test, pred_y=pred_test,df=x_test).head()

In [38]:
# train_miss = print_2x2_miss_pred(y_train, pred_train,x_train)
# test_miss =  print_2x2_miss_pred(true_y=y_test, pred_y=pred_test,df=x_test)

In [ ]:
# train_pred_df = print_2x2_pred_true(y_train, pred_train,x_train)
# test_pred_df =  print_2x2_pred_true(true_y=y_test, pred_y=pred_test,df=x_test)

In [ ]:
import lightgbm as lgb
gbm = lgb.LGBMClassifier()
# x_train.simple_bad_flg = x_train.simple_bad_flg.apply(lambda x: int(x))
# x_test.simple_bad_flg = x_test.simple_bad_flg.apply(lambda x: int(x))
# x_test.cosine_similar = x_test.cosine_similar.apply(lambda x: float(x))

x_train = res3["x_train"]
y_train = res3["y_train"]
x_test = res3["x_test"]
y_test = res3["y_test"]

gbm.fit(x_train.drop('content',axis=1),y_train)
pred_train = gbm.predict(x_train.drop('content',axis=1))
pred_test = gbm.predict(x_test.drop('content',axis=1))

# gbm.fit(x_train.drop(['content',"cosine_similary"],axis=1),y_train)
# pred_train = gbm.predict(x_train.drop(['content',"cosine_similary"],axis=1))
# pred_test = gbm.predict(x_test.drop(['content',"cosine_similary"],axis=1))
'''
train
0.8698839340256568
array([[618., 130.],
       [ 83., 712.]])
test
0.919315403422983
array([[165.,  19.],
       [ 14., 188.]])
'''

# gbm.fit(x_train_vec, y_train)
# pred_train = gbm.predict(x_train_vec)
# pred_test = gbm.predict(x_test_vec)


print("train")
print(f1_score(y_train, pred_train))
display(create_2x2_matrix(y_train, pred_train))
# print_2x2_miss_pred(y_train, pred_train,x_train).head()
print('test')

print(f1_score(y_test, pred_test))
display(create_2x2_matrix(y_test, pred_test))
# print_2x2_miss_pred(true_y=y_test, pred_y=pred_test,df=x_test).head()

In [ ]:
lgb.plot_importance(gbm)

In [ ]:
# """
# 1対その他での学習
# """


# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# def vectorize(x_train,x_test, vec):
#     x_train_vec = vec.fit_transform(x_train.content)
#     x_test_vec = vec.transform(x_test.content)
    
#     x_train_vec = pd.DataFrame(x_train_vec.todense())
#     x_test_vec = pd.DataFrame(x_test_vec.todense())
    
#     x_train_vec["comment_id"]=x_train.comment_id.values
#     x_test_vec["comment_id"]=x_test.comment_id.values
    
#     return x_train_vec,x_test_vec



# def train_and_predict(df):

#     X = df.drop("target",axis=1)
#     y = df.target

#     x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    
#     vec = TfidfVectorizer()
#     x_train_vec, x_test_vec = vectorize(x_train, x_test, vec)
#     train_content = pd.DataFrame(x_train.content.values, columns=["content"])
#     test_content  = pd.DataFrame(x_test.content.values, columns=["content"])
#     x_train.drop(["content"],axis=1,inplace=True)
#     x_test.drop(["content"],axis=1,inplace=True)
#     new_x_train = x_train.merge(x_train_vec, on="comment_id")#.drop('comment_id',axis=1)
#     new_x_test  = x_test.merge(x_test_vec, on="comment_id")#.drop('comment_id',axis=1)
#     new_x_train.drop('comment_id', axis=1, inplace=True)
#     new_x_test.drop('comment_id', axis=1, inplace=True)
#     x_train.drop(["comment_id"],axis=1,inplace=True)
#     x_test.drop(["comment_id"],axis=1,inplace=True)

#     svm = SVC(C=1.0, random_state=42)
#     vec_model = OneVsRestClassifier(svm)
#     vec_model.fit(new_x_train, y_train)
#     model = OneVsRestClassifier(svm)
#     model.fit(x_train, y_train)
#     print('fin')

#     return {
#         "pred_train_with_vec":  vec_model.predict(new_x_train),
#         "pred_test_with_vec": vec_model.predict(new_x_test),
#         "pred_train":model.predict(x_train),
#         "pred_test":model.predict(x_test),
#         "x_train":x_train,
#         "new_x_train":new_x_train,
#         "y_train":y_train,
#         "new_x_test":new_x_test,
#         "x_test":x_test,
#         "y_test":y_test,
#         "train_content":train_content,
#         "test_content":test_content,
# #         "x_train_vec":x_train_vec,
# #         "x_test_vec":x_test_vec
#     }

In [ ]:
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# import numpy as np
# '''
# mxm,ここでは3x3のマトリックスを作成する
# そこからf1値を計算する（同ディレクトリの画像参照）
# '''

# def create_3x3_matrix(true_y, pred_y):
#     matrix = np.zeros(9)

#     for t,p in zip(true_y, pred_y):
#         if t==0:
#             if p==0:matrix[0]+=1
#             if p==1:matrix[1]+=1
#             if p==2:matrix[2]+=1
#         elif t==1:
#             if p==0:matrix[3]+=1
#             if p==1:matrix[4]+=1
#             if p==2:matrix[5]+=1
#         elif t==2:
#             if p==0:matrix[6]+=1
#             if p==1:matrix[7]+=1
#             if p==2:matrix[8]+=1

#     return matrix.reshape(3,3)
# # matrix=create_3x3_matrix(y_train, pred_train)
# # matrix

In [ ]:
# df1.to_csv('../data/clean/mount_data.csv', index=False)
# df1.to_excel('../data/clean/mount_data.xlsx', index=False)
# df1 = pd.read_excel('../data/clean/mount_data.xlsx')
# df1 = df1.dropna()
# df1["type"]=df1.target
# df1 = make_bad_feature(df1, badwordlist=badwordlist, positivelist=positivelist, hinikulist=hinikulist)

In [ ]:
# df1[df1.target==0].shape[0], df1[df1.target==1].shape[0],df1[df1.target==2].shape[0]

In [ ]:
# '''
# predのキー
# dict_keys(['pred_train', 'pred_test', 'x_train', 'new_x_train', 'y_train',\
# 'new_x_test', 'x_test', 'y_test', 'train_content', 'test_content'])
# '''

# def print_3x3miss_pred(pred, test=True, true_y=np.nan, pred_y=np.nan,df=np.nan):
#     if test:
#         true_y = pred["y_test"]
#         pred_y = pred["pred_test"]
#         df     = pred["test_content"]
#     else:
#         true_y = pred["y_train"]
#         pred_y = pred["pred_train"]
#         df     = pred["train_content"]


#     idx_arr = []
#     pred_judge=[]
#     true_judge=[]
#     for i in range(len(pred_y)):
#         if pred_y[i]!=true_y.values[i]:
# #             print(pred[i], true_y.values[i])
#             idx_arr.append(i)
#             pred_judge.append(pred_y[i])
#             true_judge.append(true_y.values[i])

#     def print_str_label(num):
#         if num==0:return "ニュートラル"
#         if num==1:return "悪口"
#         if num==2:return "皮肉"
#         return np.nan
    
#     col=["content","true_label_id","true_label","pred_label_id","pred_label"]
#     miss_data = pd.DataFrame(columns=col)
#     for i,idx in enumerate(idx_arr):
#         tmpdf = pd.DataFrame(np.array([df.iloc[idx].content, true_judge[i],\
#                                           print_str_label(true_judge[i]), pred_judge[i],\
#                                         print_str_label(pred_judge[i])]).reshape(1,-1),columns=col)
#         miss_data = pd.concat([miss_data, tmpdf])
        
# #             display(df.iloc[idx].content)
# #             print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
# #             print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
# #             print("------------------------------------------------------------------------------------")
        
#     print(miss_data.shape)
#     return miss_data

In [ ]:
'''
他のベクトル変換の例

x_train,x_test, y_train, y_test = train_test_split(content, target, random_state=42)

・バイナリ
vec = CountVectorizer(binary=True)
train_and_val(x_train, x_test, y_train, y_test, vec)

・カウント
print('count')
vec = CountVectorizer(binary=False)
train_and_val(x_train, x_test, y_train, y_test, vec)


・TfIdf
vec = TfidfVectorizer()
vectorize(x_train, x_test, y_train, y_test, vec)

・bigram
vec = TfidfVectorizer(ngram_range=(1,3))
train_and_val(x_train, x_test, y_train, y_test, vec)
'''

In [ ]:
df.head(2)

In [ ]:
pred = train_and_predict(df)

In [ ]:
# print("no vectorize train")
# matrix=create_3x3_matrix(pred["y_train"], pred["pred_train"])
# print(matrix)
# print(calc_scores(matrix))
# print()
# print()
# print('no vectoizer test')
# matrix=create_3x3_matrix(pred["y_test"], pred["pred_test"])
# print(matrix)
# print(calc_scores(matrix))
# print()
# print()

# print("vectorize train")
# matrix=create_3x3_matrix(pred["y_train"], pred["pred_train_with_vec"])
# print(matrix)
# print(calc_scores(matrix))
# print()
# print()
# print('vectorize test')
# matrix=create_3x3_matrix(pred["y_test"], pred["pred_test_with_vec"])
# print(matrix)
# print(calc_scores(matrix))

In [103]:
len(res1["pred_train"]),len(res1["pred_test"])

(1543, 386)

In [104]:
len(res2["pred_train"]),len(res2["pred_test"])

(1492, 374)

In [105]:
len(res3["pred_train"]),len(res3["pred_test"])

(1492, 374)